# Downloading Forecast Data from ICON via Python API
This notebook guides you through the download process of a forecast file from the ICON numerical weather model using the `ogd_api` module. The `ogd_api` module is part of the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — an in-house developed toolkit designed to simplify working with numerical weather model data.

The data you will access is provided by MeteoSwiss as part of Switzerland’s Open Government Data (OGD) initiative.

---

## 🔍 **What You’ll Do in This Notebook**

🛰️  **Download Forecast Files**  
    Retrieve and save the requested ICON forecast files (e.g. soil temperature) to disk for offline storage using `ogd_api.download_from_ogd`.

2. 🧭  **Explore**  
    Use ecCodes to explore the forcast data file.
---

## 1. Retrieving Forecast Data
In this first part, we retrieve soil temperature forecast data from the ICON numerical weather model. To download this data, we use the `ogd_api` module from the [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) library — a convenient interface for accessing numerical weather forecasts via the [STAC (SpatioTemporal Asset Catalog) API](https://data.geo.admin.ch/api/stac/static/spec/v1/apitransactional.html#tag/Data/operation/getAsset), which provides structured access to Switzerland’s open geospatial data.

#### 📁  Browsing the STAC Catalog (Optional)

If you'd like to explore the ICON forecast datasets interactively before downloading, you can browse them directly in the STAC catalog:

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH1-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch1?.language=en)

&nbsp;&nbsp;&nbsp;&nbsp;🔗  [Browse the ICON-CH2-EPS collection](https://data.geo.admin.ch/browser/#/collections/ch.meteoschweiz.ogd-forecasting-icon-ch2?.language=en)


Below is a screenshot of the ICON-CH2-EPS collection as seen in the STAC browser interface.

![browser-ch2-resized.PNG](./images/browser-ch2-resized.PNG)

### Creating a Request
To retrieve the forecast data, we first define an API request using the `ogd_api.Request` class.
In this example the request targets lead time zero, which is the forecast's start time also called reference or initialization datetime.

>⏰ **Forecast Availability**: Forecast data will typically be available a couple of hours after the reference time — due to the model runtime and subsequent upload time. The data remains accessible for 24 hours after upload.

In [3]:
from datetime import datetime, timezone
from meteodatalab import ogd_api

today_midnight_utc = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)

req = ogd_api.Request(
    collection="ogd-forecasting-icon-ch1",
    variable="T_SO",
    reference_datetime=today_midnight_utc,
    perturbed=False,
    horizon="P0DT0H",
)

Each argument in the request serves the following purpose:

| Argument             | Description |
|----------------------|-------------|
| `collection`         | Forecast collection to use (e.g., `ogd-forecasting-icon-ch1`). |
| `variable`           | Meteorological variable of interest (`T_SO` = soil temperature). |
| `reference_datetime` | Initialization time of the forecast in **UTC** (e.g., default date of today midnight). |
| `perturbed`          | If `True`, retrieves ensemble forecast members; if `False`, returns the deterministic forecast. |
| `horizon`            | Forecast lead time, provided as either:<br>– `ISO 8601` duration string (e.g., `P0DT0H`)<br>– `datetime.timedelta` object (e.g., `datetime.timedelta(hours=0)`) |

### Downloading Data
We now send our request to the API and download the resulting dataset using the `download_from_ogd()` function. The function expects a request and a path to a directory, where the data will be stored. In this example the forecast file is stored in the folder "forecast_files" within this directory.

> 💡 **Tip**: Use temporary caching with earthkit-data to skip repeated downloads — it's auto-cleaned after the session.
> *For more details, see the [earthkit-data caching docs](https://earthkit-data.readthedocs.io/en/latest/examples/cache.html)*.

> 💡 **Hint**: If you get an error message containing "HTTPError: 403 Client Error: Forbidden for url", you may be trying to retrieve data older than 24h hours! Please adjust your requests.

In [4]:
from pathlib import Path
from earthkit.data import config
config.set("cache-policy", "temporary")

ogd_api.download_from_ogd(req, Path.cwd().joinpath("forcast_files"))

After executing the code block above, check that the download has been completed. In the folder "forecast_files" you should see three GRIB2 files and three SHA256 files:
- horizontal_constants_icon-ch1-eps.grib2
- horizontal_constants_icon-ch1-eps.sha256
- icon-ch1-eps-*today's-date*-0-t_so-ctrl.grib2
- icon-ch1-eps-*today's-date*-0-t_so-ctrl.sha256
- vertical_constants_icon-ch1-eps.grib2
- vertical_constants_icon-ch1-eps.sha256

The GRIB2 file icon-ch1-eps-*today's-date*-0-t_so-ctrl.grib2 contains the forecast data. while the other GRIB2 files store information on the heigth, longitude and latitude of the grid points. The SHA256 files are required to verify the integrity of the data.

### Verify Data Integrity

To ensure the downloaded file is not corrupted, compute its SHA-256 hash and verify it against the checksum provided in the SHA256 file.

Steps:

1. Open a terminal and generate the SHA-256 checksum of the downloaded file:

```bash
sha256sum <downloaded_GRIB_filename>
```

2. Retrieve the checksum from the SHA256 file using the following command:

```bash
cat <downloaded_SHA256_filename>
```

3. Compare the two hash values. If they match, your forecast data file is safe to use.

## 2. Explore GRIB files